# Imports

In [ ]:
from NCA.NCA import *
from NCA.utils import *
import os

# Setup

In [6]:
DEVICE = "cuda:0"
HEIGHT = 50
WIDTH = 50
CHANNELS = 16
REPETITIONS = 100
PATH_TO_IMAGE = "../Images/lizard.png" #<--- image to grow 
IMAGE_NAME = "Lizard" #<--- name of image for saving purposes 

In [11]:
path = "Trained_models/Lizard"
masking_order = [0,1,2,3,4,5,6,7,8,9,10,11,12] #<--- should be the same as the masking order in experiments 
damage_sites = [[15,7], [35,16], [15,42], [31,34]] #<--- change depending on experiment 
ncas = [ReducedCA, MaskedCA, DummyVCA]

In [12]:
reference_image,seed = get_reference_image_and_seed(PATH_TO_IMAGE, HEIGHT,WIDTH,CHANNELS)

# Path Creation for Saving Data

In [13]:
path_to_folder = "Regrow_exp_data/" + IMAGE_NAME 
if not os.path.exists(path_to_folder):
    os.makedirs(path_to_folder)
    print(f"Path: {path_to_folder} created")
else: 
    print(f"Path: {path_to_folder} already exists, all OK!")

Path: Regrow_exp_data/Lizard already exists, all OK!


In [14]:
for id, (root, subdirs, files) in enumerate(os.walk(path, topdown=True)):
    if len(files) != 0:
        loss_log = []
        files.sort(key=natural_keys)
        for idx, file in enumerate(files):
            loss_sites= []
            print(root)
            nca = ncas[id - 1](CHANNELS, hidden_n=96, mask_n=masking_order[idx])
            nca.load_state_dict(torch.load(root + "/" + file))
            nca.to(DEVICE).eval()
            for k in range(REPETITIONS):
                for dmg_site in damage_sites:
                    x = seed.clone()
                    for i in range(2048):
                        x = nca(x)
                        x = x.detach()
                        if (i != 0) and (i %1024 ==0):
                            mask = create_circular_mask([HEIGHT,WIDTH], dmg_site[1], dmg_site[0], 10)
                            x = x * ~mask[None, None, ...]
                    loss = (reference_image - x[:, :4, :, :]).pow(2).sum()
                    loss_sites.append(loss.log().item())
                print(f"Model {file}, Repetition: {k}/{REPETITIONS}, num_losses {len(loss_sites)}")
            loss_log.append(loss_sites)
        np_loss = np.asarray(loss_log)
        np.save(path_to_folder + type(nca).__name__ , np_loss )